In [16]:
!pip install librosa soundfile numpy

In [17]:
from google.colab import files

uploaded = files.upload()


Saving sanskrit_tts_output.wav to sanskrit_tts_output.wav


In [19]:
import librosa
import numpy as np
import soundfile as sf

filename = list(uploaded.keys())[0]
y_clean, sr = librosa.load(filename, sr=None)

print(f"Loaded '{filename}' with sample rate: {sr}, duration: {len(y_clean)/sr:.2f}s")


Loaded 'sanskrit_tts_output.wav' with sample rate: 24000, duration: 4.66s


In [20]:
def add_noise_to_signal(signal, noise, snr_db):
    signal_power = np.mean(signal**2)
    noise_power = np.mean(noise**2)

    desired_noise_power = signal_power / (10**(snr_db / 10))
    scale = np.sqrt(desired_noise_power / noise_power)

    return signal + noise * scale

noise = np.random.normal(0, 1, len(y_clean))

noisy_audio = add_noise_to_signal(y_clean, noise, snr_db=0)


In [21]:
sf.write("noisy_audio.wav", noisy_audio, sr)
files.download("noisy_audio.wav")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
!pip install deepfilternet

In [24]:
audio_file = "noisy_audio.wav"


In [ ]:
import torch

In [25]:
from df.enhance import enhance, init_df, load_audio, save_audio

model, df_state, _ = init_df()

device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float16" if device == "cuda" else "int8"

print(f"✅ Using device: {device}")


2025-07-13 05:09:10 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-07-13 05:09:10 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2025-07-13 05:09:10 | INFO     | DF | Initializing model `deepfilternet3`
2025-07-13 05:09:10 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-07-13 05:09:10 | INFO     | DF | Running on device cuda:0
2025-07-13 05:09:10 | INFO     | DF | Model loaded
✅ Using device: cuda


In [26]:
import os
import time

In [28]:
start = time.time()
audio, _ = load_audio(audio_file, sr=df_state.sr())
denoised_dir = "denoised"
enhanced_path = os.path.join(denoised_dir, "enhanced.wav")
enhanced = enhance(model, df_state, audio)
save_audio(enhanced_path, enhanced, df_state.sr())
print(time.time()-start)

0.33693957328796387


In [29]:
from IPython.display import Audio

Audio('denoised/enhanced.wav')

